# **Car Crashes' Severity Prediction**

**Data fields**
* Lat - Latitude of the incident
* Lng - Longitude of the incident
* Bump, Crossing, Give_Way,Junction, NoExit, RailWay, Roundabout, Stop, Amenity, Side - The characteristics of the location where the incident has taken place, several can be true at the same time. Side is the side of the street.
* State - the state from which this dataset is coming from
* Distance - the distance of the traffic jam provoked by an accident
* Timestamp - the moment when the incident has occurred.
* Severity - (Target) An indicator representing the severity of the car crash and possible impacts on the traffic. Values can range from 1 to 4, the highest value translates a highest impact.
* Severity is the target variable for that exercise. The target variable is the one we want to predict thanks to the predictive system module already present that you will build during that Programming Challenge.

# > **IMPORT LIBERIRES AND READ THE DATA**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))

print("The shape of the dataset is {}.\n\n".format(df.shape))

df.head()

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# > **EDA**

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

# > **PREPROCESSING**

In [ ]:
df["Bump"] = df["Bump"].astype(int)
df["Crossing"] = df["Crossing"].astype(int)
df["Give_Way"] = df["Give_Way"].astype(int)
df["Junction"] = df["Junction"].astype(int)
df["No_Exit"] = df["No_Exit"].astype(int)
df["Railway"] = df["Railway"].astype(int)
df["Roundabout"] = df["Roundabout"].astype(int)
df["Stop"] = df["Stop"].astype(int)
df["Amenity"] = df["Amenity"].astype(int)
df.head()

In [ ]:
f=list(df["timestamp"])                                   #convtring time 
new_time=[]
for i in range (len(f)):
    new_time.append(f[i].split()[0])
df["Time"]=new_time



In [ ]:
del df['Side']
del df['timestamp']
df

In [ ]:
df.drop(columns='ID').describe()

In [ ]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df2 = pd.read_csv(os.path.join(dataset_path, 'weather-sfcsv.csv'))

print("The shape of the dataset is {}.\n\n".format(df.shape))


In [ ]:
time=[]
d=list(df2["Day"])
m=list(df2["Month"])
y=list(df2["Year"])
for i in range(len(y)):
    s=str(y[i])+"-"+str(m[i])+"-"+str(d[i])
    time.append(s)
del df2['Day']
del df2['Year']
del df2['Month']
del df2['Hour']

df2["Time"]=time
df2

In [ ]:
df2=df2.drop_duplicates(['Time'], keep='last')
print("The shape of the Weather After removing dublicate is {}.\n\n".format(df2.shape))

In [ ]:
del df2["Selected"]
df2

In [ ]:
import xml.etree.ElementTree as et 

xtree = et.parse(os.path.join(dataset_path, 'holidays.xml'))
xroot = xtree.getroot() 

df_cols = ["date", "description"]
rows = []

for node in xroot: 
    s_description = node.find("description").text 
    s_date = node.find("date").text 
    
    
    rows.append({"description": s_description, "date": s_date})

df3 = pd.DataFrame(rows, columns = df_cols)
df3

In [ ]:
df3.rename(columns = {'date':'Time'}, inplace = True)
df3

In [ ]:
x=df.merge(df2,how="left",on=['Time'])
y=x.merge(df3,how="left",on=['Time'])
y

In [ ]:
del y["Weather_Condition"]
del y["Give_Way"]
del y["description"]
del y["Crossing"]
y

In [ ]:
y['Visibility(mi)'].fillna((y['Visibility(mi)'].mean()), inplace=True)
y['Precipitation(in)'].fillna((y['Precipitation(in)'].mean()), inplace=True)
y['Wind_Chill(F)'].fillna((y['Wind_Chill(F)'].mean()), inplace=True)
y['Temperature(F)'].fillna((y['Temperature(F)'].mean()), inplace=True)
y['Humidity(%)'].fillna((y['Humidity(%)'].mean()), inplace=True)
y['Wind_Speed(mph)'].fillna((y['Wind_Speed(mph)'].mean()), inplace=True)

In [ ]:
y

# > **SPLIT DATA**

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(y, test_size=0.2, random_state=42) # Try adding `stratify` here

X_train = train_df.drop(columns=['ID', 'Severity'])
y_train = train_df['Severity']

X_val = val_df.drop(columns=['ID', 'Severity'])
y_val = val_df['Severity']

# > **FEATURES SELECTION FOR TRAINING**

In [ ]:
# This cell is used to select the numerical features. IT SHOULD BE REMOVED AS YOU DO YOUR WORK.
X_train = X_train[['Lat', 'Lng', 'Distance(mi)','Junction','Stop']]
X_val = X_val    [['Lat', 'Lng', 'Distance(mi)','Junction','Stop']]

# > **MACHINE LEARNING MODELS**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=2, random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train)

In [ ]:
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

In [ ]:
feature_imp = pd.Series(classifier.feature_importances_,index=list(X_train.columns.values))
feature_imp

# > **READ TEST DATA**

In [ ]:
test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))
test_df.head()

# > **TEST DATA PREPROCESSING**

In [ ]:
test_df["Bump"] = test_df["Bump"].astype(int)
test_df["Crossing"] = test_df["Crossing"].astype(int)
test_df["Give_Way"] = test_df["Give_Way"].astype(int)
test_df["Junction"] = test_df["Junction"].astype(int)
test_df["No_Exit"] = test_df["No_Exit"].astype(int)
test_df["Railway"] = test_df["Railway"].astype(int)
test_df["Roundabout"] = test_df["Roundabout"].astype(int)
test_df["Stop"] = test_df["Stop"].astype(int)
test_df["Amenity"] = test_df["Amenity"].astype(int)
test_df.head()

In [ ]:
f1=list(test_df["timestamp"])                                   #convtring time 
new_time=[]
for i in range (len(f1)):
    new_time.append(f1[i].split()[0])
test_df["Time"]=new_time
test_df

In [ ]:
del test_df["timestamp"]
del test_df['Side']
test_df

In [ ]:
#test_df=test_df.drop_duplicates(['Time'], keep='last')
#print("The shape of the Weather After removing dublicate is {}.\n\n".format(test_df.shape))
#test_df

In [ ]:
X_train.columns

In [ ]:
X_test = test_df.drop(columns=['ID'])

# You should update/remove the next line once you change the features used for training
X_test = X_test[['Lat', 'Lng', 'Distance(mi)','Junction','Stop']]

y_test_predicted = classifier.predict(X_test)

test_df['Severity'] = y_test_predicted

test_df.head()

# > **SUBMISSION**

In [ ]:
test_df[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)